In [ ]:
import queue
import threading
import tempfile
import time
from collections import deque

import numpy as np
import sounddevice as sd
from scipy.io import wavfile

from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks


SAMPLE_RATE = 16000            # model expects 16 kHz
CHANNELS = 1
CHUNK_SEC = 1.0                # record chunk length
HOP_SEC = 0.5                  
PRINT_INTERVAL_SEC = 0.5       # how often to print an update
SMOOTHING_ALPHA = 0.6          

MODEL_ID = "iic/emotion2vec_plus_base"

emo = pipeline(
    task=Tasks.emotion_recognition,
    model=MODEL_ID
)

MODEL_LABELS = None

audio_q = queue.Queue(maxsize=10)
stop_flag = threading.Event()

def audio_producer():
    frame_len = int(SAMPLE_RATE * CHUNK_SEC)
    hop_len = int(SAMPLE_RATE * HOP_SEC)
    ring = deque(maxlen=frame_len)

    def callback(indata, frames, time_info, status):
        if status:
            print(f"[audio status] {status}")
        mono = indata[:, 0] if indata.ndim > 1 else indata
        ring.extend((mono * 32767).astype(np.int16))

        if len(ring) == frame_len:
            chunk = np.array(ring, dtype=np.int16)
            try:
                audio_q.put_nowait(chunk)
            except queue.Full:
                pass
            for _ in range(hop_len):
                if ring:
                    ring.popleft()

    with sd.InputStream(
        samplerate=SAMPLE_RATE,
        channels=CHANNELS,
        dtype="float32",
        callback=callback,
        blocksize=int(SAMPLE_RATE * 0.05),
    ):
        while not stop_flag.is_set():
            time.sleep(0.1)

def run_model_on_wav_bytes(int16_pcm):
    """Write a temp wav and call the ModelScope pipeline with frame granularity."""
    with tempfile.NamedTemporaryFile(suffix=".wav", delete=True) as tmp:
        wavfile.write(tmp.name, SAMPLE_RATE, int16_pcm)
        res = emo(tmp.name, granularity="frame", extract_embedding=False)
        return res
    
def infer_consumer():
    global MODEL_LABELS

    ewma_scores = None
    last_print = 0.0

    while not stop_flag.is_set():
        try:
            pcm = audio_q.get(timeout=0.2)
        except queue.Empty:
            continue

        res = run_model_on_wav_bytes(pcm)

        labels = res.get("labels")
        scores = res.get("scores")

        if labels is None or scores is None:
            detail = res.get("detail") or {}
            labels = labels or detail.get("labels")
            scores = scores or detail.get("scores")

        if labels is None or scores is None:
            print("[warn] Unexpected pipeline output keys:", res)
            continue

        if MODEL_LABELS is None:
            MODEL_LABELS = labels

        scores = np.array(scores)

        if scores.ndim == 1:
            scores = scores[None, :]

        eps = 1e-8
        scores = np.clip(scores, eps, None)
        scores = scores / (scores.sum(axis=1, keepdims=True) + eps)

        chunk_mean = scores.mean(axis=0)

        if ewma_scores is None:
            ewma_scores = chunk_mean
        else:
            ewma_scores = SMOOTHING_ALPHA * ewma_scores + (1.0 - SMOOTHING_ALPHA) * chunk_mean

        now = time.time()
        if now - last_print >= PRINT_INTERVAL_SEC:
            last_print = now
            top_idx = int(np.argmax(ewma_scores))
            top_label = MODEL_LABELS[top_idx] if MODEL_LABELS else f"class_{top_idx}"
            top_conf = float(ewma_scores[top_idx])

            top3_idx = np.argsort(-ewma_scores)[:3]
            leaderboard = ", ".join(
                f"{MODEL_LABELS[i]} {ewma_scores[i]:.2f}" for i in top3_idx
            )
            print(f"[live] {top_label:<10} {top_conf:.2f} | {leaderboard}")


if __name__ == "__main__":
    print("Loading model:", MODEL_ID)
    print("Starting mic… ")
    prod = threading.Thread(target=audio_producer, daemon=True)
    cons = threading.Thread(target=infer_consumer, daemon=True)
    prod.start()
    cons.start()
    try:
        while True:
            time.sleep(0.5)
    except KeyboardInterrupt:
        print("Stopping…")
        stop_flag.set()
        prod.join(timeout=2.0)
        cons.join(timeout=2.0)
        print("Goodbye")

In [ ]:
from modelscope.pipelines import pipeline
from modelscope.utils.constant import Tasks

inference_pipeline = pipeline(
    task=Tasks.emotion_recognition,
    model="iic/emotion2vec_plus_base")

rec_result = inference_pipeline('SOME.wav', granularity="utterance", extract_embedding=False)
print(rec_result)